In [21]:
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationBufferMemory
from langchain.schema.runnable import RunnablePassthrough

NLTK_MODEL = 'punkt'
FILES_PATH = "./files/marvels.txt"
CACHE_DIR_PATH = "./.cache/"

load_dotenv()

llm = ChatOpenAI(temperature=0.1)

memory = ConversationBufferMemory(llm=llm, max_token_limit=120, return_messages=True)

loader = UnstructuredFileLoader(FILES_PATH)
splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(separators="\n", chunk_size=500, chunk_overlap=100)
docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()
cache_dir = LocalFileStore(CACHE_DIR_PATH)
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)
vectorstore = FAISS.from_documents(docs, cached_embeddings)

retriever = vectorstore.as_retriever()

prompt = ChatPromptTemplate.from_messages([
    ("system", "When you ask questions, my responses will be based solely on the information provided below. If I don't know the answer, I'll simply say I don't know. I won't make up answers based on speculation:\n\n{context}"),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{question}")
])

def load_memory(_):
    return memory.load_memory_variables({})["history"]

chain = ({
    "context": retriever,
    "question": RunnablePassthrough(),
    "history": load_memory
} | prompt | llm)

def invoke_chain(question):
    try:
        results = chain.invoke(question)
        memory.save_context({"input": question}, {"output": results.content})
        print(results)
    except Exception as e:
        print(f"An error occurred: {e}")

invoke_chain("토니 스타크의 명대사들이 뭐지?")


content='토니 스타크의 명대사들은 다음과 같습니다:\n1. "세상을 바꾸려면 자신부터 바꿔야 해." - 아이언맨\n2. "나는 그냥 나 자신이 되는 걸 선택했어." - 아이언맨 시리즈'


In [22]:
invoke_chain("토르는?")

content='토르의 명대사들은 다음과 같습니다:\n1. "우리가 얼마나 강한지 보여주는 것은 우리가 겪는 시련이 아니라, 그것을 어떻게 극복하느냐에 달려있다." - 토르 시리즈\n2. "진정한 리더는 권력을 찾지 않지만, 그것이 필요할 때 준비되어 있어야 한다." - 토르: 라그나로크'


In [23]:
invoke_chain("토르의 무기는 뭐지?")

content='토르의 무기는 미울니르(Mjolnir)입니다.'
